In [1]:
# Test for the estimation of a single image key-points

import argparse
import grpc
import io
import matplotlib.pyplot as plt
#from importlib.machinery import SourceFileLoader
#image_generic_pb2 = SourceFileLoader("image_generic_pb2","../protos/image_generic_pb2.py").load_module()
#image_generic_pb2_grpc = SourceFileLoader("image_generic_pb2_grpc","../protos/image_generic_pb2_grpc.py").load_module()
import image_generic_pb2
import image_generic_pb2_grpc
import PIL.Image as PIL_image
import numpy as np
import cv2

In [2]:
def process_image(stub, image_path):
    print(f'Processing image: \'{image_path}\'')
    with open(image_path, 'rb') as fp:
        image_bytes = fp.read()
    request = image_generic_pb2.Image(data=image_bytes)
    return stub.process(request)


def display_image(image):
    img = PIL_image.open(io.BytesIO(image.data))
    ax = plt.gca()
    ax.imshow(img)
    plt.show()

def plot_features(img,kp):
    for p in kp:
        cv2.circle(img,(int(p[0]),int(p[1])), 3, (0,0,255), -1)
    ax =plt.gca()
    ax.imshow(img)
    plt.show()

def parse_args():
    """
    Parse arguments for test setup

    Returns:
        The arguments for the test
    """
    parser = argparse.ArgumentParser(description='Test for OpenPose gRPC Service')
    parser.add_argument(
        'image',
        help='Path to the image to send to the server')
    parser.add_argument(
        '--target',
        metavar='target',
        default='10.0.14.13:4061',
        help='Location of the tested server (defaults to localhost:8061)')
    return parser.parse_args()

In [7]:
target='localhost:4061'
image_path='image.mat'
#Para imagens muito grandes
channel_opt = [('grpc.max_send_message_length', 512 * 1024 * 1024), ('grpc.max_receive_message_length', 512 * 1024 * 1024)]
with grpc.insecure_channel(target,options=channel_opt) as channel:
    estimator_stub = image_generic_pb2_grpc.ImageGenericServiceStub(channel)
    #response = process_image(estimator_stub, image_path)
    with open(image_path, 'rb') as fp:
        image_bytes = fp.read()
    request = image_generic_pb2.Image(data=image_bytes)
    response=estimator_stub.process(request)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Exception calling application: module 'image_generic_pb2' has no attribute 'ImageImage'"
	debug_error_string = "UNKNOWN:Error received from peer ipv6:%5B::1%5D:4061 {grpc_message:"Exception calling application: module \'image_generic_pb2\' has no attribute \'ImageImage\'", grpc_status:2, created_time:"2024-06-13T00:49:51.077903+00:00"}"
>

In [ ]:
kp=np.frombuffer(response.features.data,dtype='float32').reshape(response.features.numrows,response.features.numcols)
desc=np.frombuffer(response.descriptors.data,dtype='float32').reshape(response.descriptors.numrows,response.descriptors.numcols)
img=cv2.cvtColor(cv2.imdecode(np.frombuffer(response.img.data,dtype='uint8'), cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
plot_features(img,kp)

In [ ]:
lixo=cv2.imencode('.jpg',img)

In [ ]:
im2=cv2.imdecode(lixo,cv2.IMREAD_COLOR)

In [ ]:
gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
sift = cv2.SIFT_create()
kp,desc = sift.detectAndCompute(gray,None)    


In [ ]:
output_image = cv2.drawKeypoints(gray,kp, 0, (255, 0, 0),
                                 flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
plt.imshow(output_image)
plt.show()

In [ ]:
if __name__ == '__main__':
    args = parse_args()
    target = args.target
    image_path = args.image
    with grpc.insecure_channel(target) as channel:
        estimator_stub = image_generic_pb2_grpc.ImageGenericServiceStub(channel)
        try:
            response = process_image(estimator_stub, image_path)
            display_image(response)
        except grpc.RpcError as rpc_error:
            print('An error has occurred:')
            print(f'  Error Code: {rpc_error.code()}')
            print(f'  Details: {rpc_error.details()}')

In [ ]:
a=np.array([9, 8, 7])